# Kaggle Titanic

 uniquement la fonction 

importation des librairies

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

fonction qui nettoie les données

In [23]:
def nettoyer_data(general_dataf):
    """ fonction qui pour tous Na mets la médiane, transforme M/F en 0/1 et donne des noms plus simple aux colonnes"""
    idx_nan = general_dataf[(general_dataf['Embarked'] != 'S') & (general_dataf['Embarked'] != 'C') & (general_dataf['Embarked'] != 'Q')].index
    general_dataf = general_dataf.drop(idx_nan)
    le = preprocessing.LabelEncoder()
    le.fit(general_dataf['Embarked'])
    general_dataf['Embarked']=le.transform(general_dataf["Embarked"])
    le.fit(general_dataf['Sex'])
    general_dataf['Sex']=le.transform(general_dataf['Sex'])
    general_dataf['Age']=general_dataf['Age'].fillna(general_dataf['Age'].median())
    general_dataf=general_dataf.rename(columns={'SibSp':'Nb_freresoeur_conjoint','Parch':'nb_parentenfants','Fare':'prix'})
    general_dataf['prix']=general_dataf['prix'].fillna(general_dataf['prix'].median())
    
    return general_dataf



fonction qui selectionne les x et y

In [24]:
def select(data_train,data_test,list_col,) :
    x=data_train.loc[:,list_col]
    y=data_train.loc[:,['Survived']]
    x_test=data_test.loc[:,list_col]
    return (x,y,x_test)

fonction transforme age en classe

In [25]:

def age_cat(age):
    if age<=3:
        return 0
    elif age<= 12 and age>3:
        return 1
    elif age<= 18 and age>12:
        return 2
    elif age<= 40 and age>18:
        return 3
    elif age<= 60 and age>40:
        return 4
    elif age>60:
        return 5

def col_cat(data):
    data['cat_age'] = data['Age'].map(age_cat)
    return data
 

fonction générale

In [33]:
def Kaggle_titanic(fich_train,fich_test,colonne_a_tester,name_csv,model_choose):
    general_data = pd.read_csv(fich_train)
    dt=nettoyer_data(general_data) 
    dt_test= pd.read_csv(fich_test)
    dt_test=nettoyer_data(dt_test)
    dt=col_cat(dt)
    dt_test=col_cat(dt_test)
    # recupèrer les id
    id_test=dt_test.loc[:,['PassengerId']]
    Xt,Yt,y=select(dt,dt_test,colonne_a_tester)
    from sklearn.cross_validation import train_test_split
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(Xt,Yt, test_size=0.3, random_state=0)
    # machine learning
    print("Avec la méthode "+str(model_choose))
    model = model_choose
    model.fit(X_train, y_train)
    # check the accuracy on the training set
    accuracy_train=model.score(X_train, y_train)
    print("the accurucy on the training set is %f"%(accuracy_train))
    predi=model.predict(X_test)
    accuracy_test=model.score(X_test,y_test)
    print (dt_test)
    print("the accurucy on the tesx set is %f"%(accuracy_test))
    Survive=model.predict(dt_test)
    PassengerId = dt_test['PassengerId'].tolist()
    data = pd.DataFrame( {'PassengerId' :PassengerId, 'Survived' : Survive} )
    data = data.set_index('PassengerId')
    data.to_csv(name_csv)
    return True
Kaggle_titanic("train.csv","test.csv",['Age',"cat_age","prix","Embarked"],"fonct_RL.csv",LogisticRegression())
Kaggle_titanic("train.csv","test.csv",['Age',"Sex","prix","Embarked"],"fonct_GNB.csv",LogisticRegression())
Kaggle_titanic("train.csv","test.csv",['Age',"Sex","prix","Embarked"],"fonct_Treeba.csv",tree.DecisionTreeClassifier())


Avec la méthode LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
the accurucy on the training set is 0.688103
     PassengerId  Pclass                                               Name  \
0            892       3                                   Kelly, Mr. James   
1            893       3                   Wilkes, Mrs. James (Ellen Needs)   
2            894       2                          Myles, Mr. Thomas Francis   
3            895       3                                   Wirz, Mr. Albert   
4            896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5            897       3                         Svensson, Mr. Johan Cervin   
6            898       3                               Connolly, Miss. Kate   
7            899       2                  

ValueError: could not convert string to float: C105